# DSC 450: Take home Final

## 1. We will use a full day worth of tweets as an input (there are total of 4.4M tweets in this file, but you only need to read 1M): 
    http://rasinsrv07.cstcis.cti.depaul.edu/CSC455/OneDayOfTweets.txt

### a. Create a 3rd table incorporating the Geo table (in addition to tweet and user tables that you already have from HW4 and HW5) and extend your schema accordingly. 
    You do not need to use ALTER TABLE, it is sufficient to just re-make your schema.
    You will need to generate an ID for the Geo table primary key (you may use any value or reasonable combination of values as long as it is unique) for that table and link it to the Tweet table (foreign key should be in the Tweet). In addition to the primary key column, the geo table should have at least the “type”, “longitude” and “latitude” columns.

### b. Use python to download from the web and save to a local text file (not into database yet, just to text file) at least 1,000,000 lines worth of tweets. 
    Test your code with fewer rows first and only time it when you know it works. Report how long did it take.

In [1]:
import sqlite3
conn = sqlite3.connect('dsc450_Final_web.db')
c = conn.cursor()
                       
usertable = '''CREATE TABLE user_id(
    id         int,
    name       text,
    screen_name  text,
    description  text,
    friends_count  int,
    
    PRIMARY KEY (id)
);'''

geotable = '''CREATE TABLE geo(
    id int,
    type text,
    longitude text,
    latitude text,
    
    PRIMARY KEY (id)
);'''

tweettable = '''CREATE TABLE tweet(
    id_str  int,
    created_at text,
    text  text,
    source  text, 
    in_reply_to_user_id int,
    in_reply_to_screen_name  text,
    in_reply_to_status_id int,
    retweet_count int,
    contributors  text,
    geo_id text,
    user_id  int,
    
    CONSTRAINT Tweets_PK  PRIMARY KEY (id_str),

    CONSTRAINT Tweets_FK1 FOREIGN KEY (user_id)
    REFERENCES User(id),
    
    CONSTRAINT Tweets_FK2 FOREIGN KEY (geo_id)
    REFERENCES geo(id)
);'''

    
try:
    c.execute('drop table tweet')
    c.execute('drop table user_id')
    c.execute('drop table geo')
except:
    pass

import sys
import json
import urllib
import time

c.execute(usertable)
c.execute(geotable)
c.execute(tweettable)

startweb = time.time()
webFD=urllib.request.urlopen ("http://rasinsrv07.cstcis.cti.depaul.edu/CSC455/OneDayOfTweets.txt")
d = open('OneDayOfTweets1.txt','w')

user_list, tweet_list, geo_list = [], [], [] 
geoid = 0

for i in range(1000000):
    alltweet=webFD.readline()
    d.write(alltweet.decode('utf8'))
    try:
        tweet = alltweet
        tdict = json.loads(tweet)
    except:
        pass
    else:
        newRowUser = [] # hold individual values of to-be-inserted row for user table
        userKeys = ['id', 'name', 'screen_name', 'description', 'friends_count']
        userDict = tdict['user']
        for key in userKeys: # For each dictionary key we want
            if userDict[key] == 'null' or userDict[key] == '':
                newRowUser.append(None)   # proper NULL
            else:
                newRowUser.append(userDict[key]) # use value as-is
        user_list.append((newRowUser))
            
    
        newRowTweet = [] # hold individual values of to-be-inserted row
        tweetKeys = ['id_str','created_at','text','source','in_reply_to_user_id', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'retweet_count', 'contributors']
        geoid += 1
        for key in tweetKeys:
            if tdict[key] == 'null' or tdict[key] == '':
                newRowTweet.append(None)   # proper NULL
            else:
                newRowTweet.append(tdict[key]) # use value as-is
        userDict = tdict['user'] # This the the dictionary for user information
        if tdict['geo']:
            newRowTweet.append(geoid) # Take the point type
            geo_list.append((geoid,tdict['geo']['type'],tdict['geo']['coordinates'][0],tdict['geo']['coordinates'][1]))
        else:
            newRowTweet.append(None)
        newRowTweet.append(userDict['id']) # User id/ foreign key
        tweet_list.append((newRowTweet))
        
d.close()

stopweb = time.time()
print("diff "+str(stopweb-startweb))
startinsert = time.time()
c.executemany('INSERT OR IGNORE INTO tweet VALUES(?,?,?,?,?,?,?,?,?,?,?)',tweet_list) 
c.executemany('INSERT OR IGNORE INTO user_id VALUES(?,?,?,?,?)',user_list)
c.executemany('INSERT OR IGNORE INTO geo VALUES(?,?,?,?)',geo_list)
stopinsert = time.time()

diff 599.9838438034058


### c. Repeat what you did in part-b, but instead of saving tweets to the file, populate the 3- table schema that you created in SQLite. 
    Be sure to execute commit and verify that the data has been successfully loaded (report loaded row counts for each of the 3 tables). How long did this step take?

In [3]:
start = time.time()
print(c.execute('select count(*) from user_id').fetchall())
print(c.execute('select count(*) from tweet').fetchall())
print(c.execute('select count(*) from geo').fetchall())
print(c.execute('select count(*) from tweet where geo_id != "None"').fetchall())
print(c.execute('select count(*) from tweet where geo_id is NULL').fetchall())
stop = time.time()
print("diff "+str(stop-start))

[(855488,)]
[(999553,)]
[(24320,)]
[(24315,)]
[(975238,)]
diff 9.303666830062866


### d.	Use your locally saved tweet file (created in part-b) to repeat the database population step from part-c. 
    That is, load 1,000,000 tweets into the 3-table database using your saved file with tweets (do not use the URL to read twitter data). 
    How does the runtime compare with part-c?

In [4]:
import sqlite3
import json
import time


confile = sqlite3.connect('dsc450_Final_file2.db')
cfile= confile.cursor()
                       
usertable = '''CREATE TABLE user_id(
    id         int,
    name       text,
    screen_name  text,
    description  text,
    friends_count  int,
    
    PRIMARY KEY (id)
);'''

geotable = '''CREATE TABLE geo(
    id int,
    type text,
    longitude text,
    latitude text,
    
    PRIMARY KEY (id)
);'''

tweettable = '''CREATE TABLE tweet(
    id_str  int,
    created_at text,
    text  text,
    source  text, 
    in_reply_to_user_id int,
    in_reply_to_screen_name  text,
    in_reply_to_status_id int,
    retweet_count int,
    contributors  text,
    geo_id text,
    user_id  int,
    
    CONSTRAINT Tweets_PK  PRIMARY KEY (id_str),

    CONSTRAINT Tweets_FK1 FOREIGN KEY (user_id)
    REFERENCES User(id),
    
    CONSTRAINT Tweets_FK2 FOREIGN KEY (geo_id)
    REFERENCES geo(id)
);'''

try:
    cfile.execute('drop table tweet')
    cfile.execute('drop table user_id')
    cfile.execute('drop table geo')
except:
    pass

cfile.execute(tweettable)
cfile.execute(usertable)
cfile.execute(geotable)

startfile = time.time()
file = open('OneDayOfTweets1.txt','r')
user_list_file, tweet_list_file, geo_list_file = [], [], [] 
geoid_infile = 0


for i in range(1000000):
    alltweets=file.readline()
    try:
        tweets = alltweets
        tdict = json.loads(tweets)
  
    except:
        pass
    else:
        newRowUser = [] # hold individual values of to-be-inserted row for user table
        userKeys = ['id', 'name', 'screen_name', 'description', 'friends_count']
        userDict = tdict['user']
        for key in userKeys: # For each dictionary key we want
            if userDict[key] == 'null' or userDict[key] == '':
                newRowUser.append(None)   # proper NULL
            else:
                newRowUser.append(userDict[key]) # use value as-is
        user_list_file.append((newRowUser))
            
    
        newRowTweet = [] # hold individual values of to-be-inserted row
        tweetKeys = ['id_str','created_at','text','source','in_reply_to_user_id', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'retweet_count', 'contributors']
        geoid_infile += 1
        for key in tweetKeys:
            if tdict[key] == 'null' or tdict[key] == '':
                newRowTweet.append(None)   # proper NULL
            else:
                newRowTweet.append(tdict[key]) # use value as-is
        userDict = tdict['user'] # This the the dictionary for user information
        if tdict['geo']:
            newRowTweet.append(geoid_infile) # Take the point type
            geo_list_file.append((geoid_infile,tdict['geo']['type'],tdict['geo']['coordinates'][0],tdict['geo']['coordinates'][1]))
        else:
            newRowTweet.append(None)
        newRowTweet.append(userDict['id']) # User id/ foreign key
        tweet_list_file.append((newRowTweet))
        
file.close()
stopfile = time.time()
print("diff "+str(stopfile-startfile))
startfileinsert = time.time()
cfile.executemany('INSERT OR IGNORE  INTO user_id VALUES(?,?,?,?,?)',user_list_file)
cfile.executemany('INSERT OR IGNORE  INTO tweet VALUES(?,?,?,?,?,?,?,?,?,?,?)',tweet_list_file) 
cfile.executemany('INSERT OR IGNORE  INTO geo VALUES(?,?,?,?)',geo_list_file) 
stopfileinsert = time.time()
print("diff "+str(stopfileinsert-startfileinsert))

start = time.time()
print(cfile.execute('select count(*) from user_id').fetchall())
print(cfile.execute('select count(*) from tweet').fetchall())
print(cfile.execute('select count(*) from geo').fetchall())
print(cfile.execute('select count(*) from tweet where geo_id != "None"').fetchall())
print(cfile.execute('select count(*) from tweet where geo_id is NULL').fetchall())
stop = time.time()
print("diff "+str(stop-start))

diff 91.0920569896698
diff 18.501519680023193
[(855488,)]
[(999553,)]
[(24320,)]
[(24315,)]
[(975238,)]
diff 1.722151756286621


###  e. Re-run the previous step with a batching size of 1000 (i.e. by inserting 1000 rows at a time with executemany).

    How does the runtime compare when batching is used?

In [5]:
conbatch = sqlite3.connect('dsc450_Final_batching.db')
cbatching= confile.cursor()
                       
usertable = '''CREATE TABLE user_id(
    id         int,
    name       text,
    screen_name  text,
    description  text,
    friends_count  int,
    
    PRIMARY KEY (id)
);'''

geotable = '''CREATE TABLE geo(
    id int,
    type text,
    longitude text,
    latitude text,
    
    PRIMARY KEY (id)
);'''

tweettable = '''CREATE TABLE tweet(
    id_str  int,
    created_at text,
    text  text,
    source  text, 
    in_reply_to_user_id int,
    in_reply_to_screen_name  text,
    in_reply_to_status_id int,
    retweet_count int,
    contributors  text,
    geo_id text,
    user_id  int,
    
    CONSTRAINT Tweets_PK  PRIMARY KEY (id_str),

    CONSTRAINT Tweets_FK1 FOREIGN KEY (user_id)
    REFERENCES User(id),
    
    CONSTRAINT Tweets_FK2 FOREIGN KEY (geo_id)
    REFERENCES geo(id)
);'''

try:
    cbatching.execute('drop table tweet')
    cbatching.execute('drop table user_id')
    cbatching.execute('drop table geo')
except:
    pass
import sys
import json
import urllib
import time

cbatching.execute(usertable)
cbatching.execute(geotable)
cbatching.execute(tweettable)

startweb = time.time()
webFD=urllib.request.urlopen ("http://rasinsrv07.cstcis.cti.depaul.edu/CSC455/OneDayOfTweets.txt")
user_list_b, tweet_list_b, geo_list_b = [], [], [] 
geoid_b = 0
loadCounter = 0

for i in range(1000000):
    alltweet=webFD.readline()
    if i % 1000 == 0: # Print a message every 100th tweet read
        print ("Processed " + str(i) + " tweets")
        
    try:
        tweet = alltweet
        tdict = json.loads(tweet)
        loadCounter = loadCounter + 1
    except:
        pass
    else:
        newRowUser = [] # hold individual values of to-be-inserted row for user table
        userKeys = ['id', 'name', 'screen_name', 'description', 'friends_count']
        userDict = tdict['user']
        for key in userKeys: # For each dictionary key we want
            if userDict[key] == 'null' or userDict[key] == '':
                newRowUser.append(None)   # proper NULL
            else:
                newRowUser.append(userDict[key]) # use value as-is
        if loadCounter < 1000: # Batching 50 at a time
            user_list.append((newRowUser))
        else:
            cbatching.executemany('INSERT OR IGNORE INTO user_id VALUES(?,?,?,?,?)',user_list)
            user_list_b = [] # Reset the list of batched tweets
    
            
        newRowTweet = [] # hold individual values of to-be-inserted row
        tweetKeys = ['id_str','created_at','text','source','in_reply_to_user_id', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'retweet_count', 'contributors']
        geoid_b += 1
        for key in tweetKeys:
            if tdict[key] == 'null' or tdict[key] == '':
                newRowTweet.append(None)   # proper NULL
            else:
                newRowTweet.append(tdict[key]) # use value as-is
        userDict = tdict['user'] # This the the dictionary for user information
        if tdict['geo']:
            newRowTweet.append(geoid) # Take the point type
            if loadCounter < 1000: # Batching 50 at a time
                geo_list_b.append((geoid,tdict['geo']['type'],tdict['geo']['coordinates'][0],tdict['geo']['coordinates'][1]))
        else:
            newRowTweet.append(None)
        newRowTweet.append(userDict['id']) # User id/ foreign key
        
        if loadCounter < 1000: # Batching 50 at a time
            tweet_list_b.append((newRowTweet))
        else:
            cbatching.executemany('INSERT OR IGNORE INTO tweet VALUES(?,?,?,?,?,?,?,?,?,?,?)',tweet_list_b)
            cbatching.executemany('INSERT OR IGNORE INTO geo VALUES(?,?,?,?)',geo_list_b)
            loadCounter = 0
            tweet_list_b = []
            geo_list_b = []# Reset the list of batched users
d.close()

stopweb = time.time()
print("diff "+str(stopweb-startweb))
startinsert = time.time()
cbatching.executemany('INSERT OR IGNORE INTO tweet VALUES(?,?,?,?,?,?,?,?,?,?,?)',tweet_list_b) 
cbatching.executemany('INSERT OR IGNORE INTO user_id VALUES(?,?,?,?,?)',user_list_b)
cbatching.executemany('INSERT OR IGNORE INTO geo VALUES(?,?,?,?)',geo_list_b)
stopinsert = time.time()
print("diff "+str(stopinsert-startinsert))

start = time.time()
print(cbatching.execute('select count(*) from user_id').fetchall())
print(cbatching.execute('select count(*) from tweet').fetchall())
print(cbatching.execute('select count(*) from geo').fetchall())
print(cbatching.execute('select count(*) from tweet where geo_id != "None"').fetchall())
print(cbatching.execute('select count(*) from tweet where geo_id is NULL').fetchall())
stop = time.time()
print("diff "+str(stop-start))


Processed 0 tweets
Processed 1000 tweets
Processed 2000 tweets
Processed 3000 tweets
Processed 4000 tweets
Processed 5000 tweets
Processed 6000 tweets
Processed 7000 tweets
Processed 8000 tweets
Processed 9000 tweets
Processed 10000 tweets
Processed 11000 tweets
Processed 12000 tweets
Processed 13000 tweets
Processed 14000 tweets
Processed 15000 tweets
Processed 16000 tweets
Processed 17000 tweets
Processed 18000 tweets
Processed 19000 tweets
Processed 20000 tweets
Processed 21000 tweets
Processed 22000 tweets
Processed 23000 tweets
Processed 24000 tweets
Processed 25000 tweets
Processed 26000 tweets
Processed 27000 tweets
Processed 28000 tweets
Processed 29000 tweets
Processed 30000 tweets
Processed 31000 tweets
Processed 32000 tweets
Processed 33000 tweets
Processed 34000 tweets
Processed 35000 tweets
Processed 36000 tweets
Processed 37000 tweets
Processed 38000 tweets
Processed 39000 tweets
Processed 40000 tweets
Processed 41000 tweets
Processed 42000 tweets
Processed 43000 tweets
P

ConnectionResetError: [Errno 54] Connection reset by peer

## 2. Write and execute SQL queries to do the following. 
    Don’t forget to report the running times in each part and the code you used.

## a.
### i. Find tweets where tweet id_str contains “55” or “88” anywhere in the column

In [6]:
start = time.time()
for i in cfile.execute('select * from tweet where id_str like "%55%" or  id_str like "%88%"').fetchall():
    print(i)
stop = time.time()
print("diff "+str(stop-start))

(471803285738106880, 'Thu May 29 00:00:43 +0000 2014', 'Mucho la Plop esto, la Plop aquello, pero de los viernes es la fiesta con la gente más linda. \nEn las otras vienen directo de la frontera.', 'web', None, None, None, 0, None, None, 38950479)
(471803285742288896, 'Thu May 29 00:00:43 +0000 2014', 'RT @beyles66: cant wait for next year with @Harrison_BHS and @ThePurbalite', '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', None, None, None, 0, None, None, 329489005)
(471803285762883585, 'Thu May 29 00:00:43 +0000 2014', '【定時】\nマルキュウマルマル\n\n学校の日は\n これから始まる授業に萎えますね…', '<a href="http://twittbot.net/" rel="nofollow">twittbot.net</a>', None, None, None, 0, None, None, 174088334)
(471803285771255808, 'Thu May 29 00:00:43 +0000 2014', 'RT @rogergzz: #HoyNoMePuedoLevantarConRoger o si? RT http://t.co/BC4oeRhvpp', 'web', None, None, None, 0, None, None, 318452532)
(471803285762887680, 'Thu May 29 00:00:43 +0000 2014', 'RT @RelatableQuote: I laughed way 

(471803923288686592, 'Thu May 29 00:03:15 +0000 2014', '5月29日(木) 9:00\u3000今後の天気と最高/低気温\u3000■西部(名古屋) 曇後晴 30℃/17℃\u3000■東部(豊橋) 晴れ 26℃/17℃\u3000', '<a href="http://twitter.com/" rel="nofollow">Aichi_TW</a>', None, None, None, 0, None, None, 543220664)
(471803923305885696, 'Thu May 29 00:03:15 +0000 2014', 'RT @Cristiano: Relaxing time. http://t.co/raVxPbnQoV', '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', None, None, None, 0, None, None, 1350867043)
(471803923297087488, 'Thu May 29 00:03:15 +0000 2014', "My life is easy when I'm not around u", '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', None, None, None, 0, None, None, 809744226)
(471803923292889089, 'Thu May 29 00:03:15 +0000 2014', "@JeffNuich Let's hope he gets the support he needs at the net", '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 390325201, 'JeffNuich', 471803534552600600, 0, None, None, 611312402)
(4718

(471804221088862208, 'Thu May 29 00:04:26 +0000 2014', 'Mañana te hablo guapa, y q sepas q t sigo esperando — nonono o me hablaa hoy o nada eh http://t.co/HPHNBeVcNR', '<a href="http://ask.fm/" rel="nofollow">Ask.fm</a>', None, None, None, 0, None, None, 1563986066)
(471804221096869888, 'Thu May 29 00:04:26 +0000 2014', 'Ta frio, mamãe http://t.co/ptphWD9E5N', '<a href="http://instagram.com" rel="nofollow">Instagram</a>', None, None, None, 0, None, None, 100295716)
(471804221088489472, 'Thu May 29 00:04:26 +0000 2014', 'RT @SantapanMinda: Mengetahui perkara yang betul tidak memadai dan bermakna jika tidak melakukan perkara yang betul. #renungan', 'web', None, None, None, 0, None, None, 981670808)
(471804221084274688, 'Thu May 29 00:04:26 +0000 2014', 'RT @utsukushisahi: 好きって言われれば言われるほど、不安になるときだってあるよ', '<a href="https://twitter.com/koihikiyoseru" rel="nofollow">長谷川６５</a>', None, None, None, 0, None, None, 2487690386)
(471804225291554816, 'Thu May 29 00:04:27 +0000 2014', '#FutbolTotalDI

(471804774740799488, 'Thu May 29 00:06:38 +0000 2014', 'RT @sccerprobs: Retweet if you love playing soccer in the rain http://t.co/h0XjfVu8wc', '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', None, None, None, 0, None, None, 1706145126)
(471804778931314688, 'Thu May 29 00:06:39 +0000 2014', "Liberals, democrats, socialists, marxists, and communists - they're all the same. #VA", '<a href="http://02cd446.netsolhost.com/resources/beacon.html" rel="nofollow">Telling My Story </a>', None, None, None, 0, None, None, 1069549602)
(471804778927116288, 'Thu May 29 00:06:39 +0000 2014', "Hussein Obama's war against electricity claims more casualties: http://t.co/ni4Hjbo5hs #VA", '<a href="http://02cd446.netsolhost.com/resources/beacon.html" rel="nofollow">Telling My Story </a>', None, None, None, 0, None, None, 1069549602)
(471804778935517184, 'Thu May 29 00:06:39 +0000 2014', 'RT @RooGomez42: @Dahyana_398 de nada jajajajjaa :)', '<a href="https://mobile.twit

(471805546480160768, 'Thu May 29 00:09:42 +0000 2014', '@CNBJohn90 mwoyaaaaaa!!! &gt;.&lt;', '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 1095215490, 'CNBJohn90', 471805404054179840, 0, None, None, 1594379106)
(471805546492739584, 'Thu May 29 00:09:42 +0000 2014', 'RT @a7lanat2: 55 ➊ #عاجل✔ ➋ #لزيادة_متابعينك✔➌ #فرصتك✔➍ #فولومي@a7lanat2✔➎ #فولوباك✔➏ #اضافة_من_عمل_ريتويت✔➐ #تابعني ✔#FF #KSA http://t.co/…', '<a href="http://a33n.com" rel="nofollow">testapplicationlolo</a>', None, None, None, 0, None, None, 2434608828)
(471805546463363072, 'Thu May 29 00:09:42 +0000 2014', 'RT @hanshintigersjp: 4回裏、マートン！牧田から勝ち越しタイムリーヒット！阪神2-1西武。http://t.co/RSnTE7ytDL  #hanshin #tigers #阪神 #タイガース', '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', None, None, None, 0, None, None, 2228255347)
(471805546496946176, 'Thu May 29 00:09:42 +0000 2014', 'RT @mattm032: #NHL15Bergeron 4143', '<a href="http://twitter.com/download/android" re

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### ii.	Find how many unique values are there in the “in_reply_to_user_id” column

In [7]:
start=time.time()
print(cfile.execute('''select count(*) from (
                        select distinct in_reply_to_user_id from tweet)''').fetchall())
stop = time.time()
print("diff "+str(stop-start))

[(8577,)]
diff 0.023343324661254883


### iii.	Find the tweet(s) with the shortest, longest and average length text message.


In [8]:
start=time.time()
for ltwe in cfile.execute('''select text from tweet 
                      where length(text) = (select min(length(text))from tweet)''').fetchall():
    print(ltwe)
    
print(cfile.execute('''select min(length(text))from tweet''').fetchall())
    
for stwe in cfile.execute('''select text from tweet 
                      where length(text) = (select max(length(text))from tweet)''').fetchall():
    print(stwe)

for avgtwe in cfile.execute('''select text from tweet 
                      where length(text) = (select round(avg(length(text)), -1) from tweet)''').fetchall():
    print(avgtwe)
stop = time.time()
print("diff "+str(stop-start))

('q',)
('😒',)
('😑',)
('🅾',)
('？',)
('.',)
('😑',)
('こ',)
('😶',)
('ま',)
('😔',)
('Ê',)
('😻',)
('😕',)
('👀',)
('😭',)
('0',)
('X',)
('📴',)
('I',)
('P',)
('.',)
('😪',)
('😒',)
('ツ',)
('￦',)
[(1,)]
("RT @JesuOtaku: wait that's not enough &gt;'s. F/Z &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;…",)
('RT @bysuwari0: çocuklar karşılıklı oturup favlaşmışlar bize laf düşmez',)
('RT @amellywood: Feels like a good night.\nGo #Kings Go!\n2-0 win.\n(DTTT)',)
('@wilmerdon Yes !!! Travel for Presence &amp; suzernaity (expeditions )',)
('@gauts13   Oh so what did you think about the John Moore hit on Wiese?',)
('RT @hanyukunnmedal: 羽生君がJump Around出てた時の画像ください https://t.co/RJsvr5sbi5',)
('م

### iv.	Find the average longitude and latitude value for each user name.

In [ ]:
start = time.time()
for avgtwe in cfile.execute('''select user_id.name, geo.longitude, geo.latitude from tweet, geo, user_id
where geo.longitude = (select avg(longitude) from geo)and geo.latitude = (select avg(latitude) from geo) and tweet.geo_id = geo.id''').fetchall():
    print(avgtwe)
stop = time.time()
print("diff "+str(stop-start))

### v.	Find how many known/unknown locations there were in total (e.g., 50,000 known, 950,000 unknown,  5% locations are available)


In [ ]:
start = time.time()
a = cfile.execute('''select count(*)from tweet where geo_id is not null''').fetchall()
stop = time.time()
print(a)
print("diff "+str(stop-start))

start = time.time()
a=cfile.execute('''select count(*)from tweet where geo_id is not null''').fetchall()[0][0]/cfile.execute(
'''select count(*) from tweet''').fetchall()[0][0]
stop = time.time()
print('{0:.4f}%'.format((a* 100)))
print("diff "+str(stop-start))

### vi.	Re-execute the query in part iv) 10 times and 100 times and measure the total runtime (just re-run the same exact query multiple times using a for-loop).
    Does the runtime scale linearly? (i.e., does it take 10X and 100X as much time?)

In [ ]:
start1 = time.time()
for i in range(10):
    for avgtwe in cfile.execute('''select user_id.name, geo.longitude, geo.latitude from tweet, geo, user_id
where geo.longitude = (select avg(longitude) from geo)and geo.latitude = (select avg(latitude) from geo) and tweet.geo_id = geo.id''').fetchall():
        print(avgtwe)
stop1 = time.time()
print("diff "+str(stop1-start1))

start2 = time.time()
for i in range(100):
    for avgtwe in cfile.execute('''select user_id.name, geo.longitude, geo.latitude from tweet, geo, user_id
where geo.longitude = (select avg(longitude) from geo)and geo.latitude = (select avg(latitude) from geo) and tweet.geo_id = geo.id''').fetchall():
        print(avgtwe)
stop2 = time.time()
print("diff "+str(stop2-start2))

print("times "+str((stop2-start2)/(stop1-start1)))

## b. 
    Write python code that is going to read the locally saved tweet data file from 1-b and perform the equivalent computation for parts 2-i and 2-ii only. How does the runtime compare to the SQL queries?

### 2i.

In [ ]:
start = time.time()
openfile = open('OneDayOfTweets1.txt','r')
tweet_list = []
for i in range(1000000):
    alltweet=openfile.readline()
    try:
        tweet = alltweet
        tdict = json.loads(tweet)
    except:
        pass
    else:
        newRowTweet = [] # hold individual values of to-be-inserted row
        tweetKeys = ['id_str','created_at','text','source','in_reply_to_user_id', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'retweet_count', 'contributors']
        geoid += 1
        for key in tweetKeys:
            if tdict[key] == 'null' or tdict[key] == '':
                newRowTweet.append(None)   # proper NULL
            else:
                newRowTweet.append(tdict[key]) # use value as-is
        userDict = tdict['user'] # This the the dictionary for user information
        if tdict['geo']:
            newRowTweet.append(geoid) # Take the point type
        else:
            newRowTweet.append(None)
        newRowTweet.append(userDict['id']) # User id/ foreign key
        tweet_list.append((newRowTweet))

for x in range(len(tweet_list)):
    if '55' in tweet_list[x][0] or '88' in tweet_list[x][0]:
        print(tweet_list[x])
stop = time.time()
print("diff "+str(stop-start))

#print(cfile.execute('''select  in_reply_to_user_id from tweet''').fetchall())
#print(cfile.execute('''select * from (
#                       select distinct in_reply_to_user_id from tweet)''').fetchall())
#print(cfile.execute('''select count(*) from (
#                     select distinct in_reply_to_user_id from tweet)''').fetchall())
#print(cfile.execute('''select * from tweet
#                     where tweet.in_reply_to_user_id =93420016''').fetchall())


### 2ii

In [ ]:
start = time.time()
openfile = open('OneDayOfTweets1.txt','r')
tweet_list = []
replylst=[]

for i in range(100000):
    alltweet=openfile.readline()
    try:
        tweet = alltweet
        tdict = json.loads(tweet)
    except:
        pass
    else:
        newRowTweet = [] # hold individual values of to-be-inserted row
        tweetKeys = ['id_str','created_at','text','source','in_reply_to_user_id', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'retweet_count', 'contributors']
        geoid += 1
        for key in tweetKeys:
            if tdict[key] == 'null' or tdict[key] == '':
                newRowTweet.append(None)   # proper NULL
            else:
                newRowTweet.append(tdict[key]) # use value as-is
        userDict = tdict['user'] # This the the dictionary for user information
        if tdict['geo']:
            newRowTweet.append(geoid) # Take the point type
        else:
            newRowTweet.append(None)
        newRowTweet.append(userDict['id']) # User id/ foreign key
        tweet_list.append((newRowTweet))

for x in range(len(tweet_list)):
    if tweet_list[x][4] != None:
        if tweet_list[x][4] not in replylst:
            replylst.append(tweet_list[x][4])
withoutnone = len(replylst)
total= withoutnone +1  
print(total)

stop = time.time()
print("diff "+str(stop-start))

## 3.

### a.	Export the contents of the User table from a SQLite table into a sequence of INSERT statements within a file. This is very similar to what you already did in Assignment 4. However, you have to add a unique ID column which has to be a string (you cannot use numbers). 
    Hint: you can replace digits with letters, e.g., chr(ord('a')+1) gives you a 'b' and chr(ord('a')+2) returns a 'c'


In [ ]:
start = time.time()
def generateInsertStatements(table):
    data= c.execute('select * from '+ table).fetchall()
    d = open('inserts.txt','a') 
    a=0
    for value in data:
        a += 1
        unique_id = chr(ord('a')+100+a) 
        InsertStatements='INSERT INTO '+ table+' VALUES' + str((unique_id,) + (value)) +'\n'
        d.write(InsertStatements)
generateInsertStatements('user_id')
stop = time.time()
print("diff "+str(stop-start))
a = open("inserts.txt", 'r')# open file in read mode 
print("the file contains:") 
print(a.read())


### b. Create the same collection of INSERT for the User table by reading data from the local tweet file that you have saved earlier.
    How do these compare in runtime? Which method was faster?

In [ ]:
start = time.time()
def generateInsertStatementsfromtext(table):
    openfile = open('OneDayOfTweets1.txt','r')
    d = open('insertsfromtext.txt','a') 
    user_list = ()
    a=0
    for i in range(1000000):
        alltweet=openfile.readline()
        try:
            tweet = alltweet
            tdict = json.loads(tweet)
        except:
            pass
        else:
            newRowUser = () # hold individual values of to-be-inserted row for user table
            userKeys = ['id', 'name', 'screen_name', 'description', 'friends_count']
            userDict = tdict['user']
            for key in userKeys: # For each dictionary key we want
                if userDict[key] == 'null' or userDict[key] == '':
                    newRowUser += (None,)  # proper NULL
                else:
                    newRowUser +=(userDict[key],) # use value as-is
        a += 1
        unique_id = chr(ord('a')+100+a) 
        InsertStatements='INSERT INTO '+ table+' VALUES' + str((unique_id,) + (newRowUser)) +'\n'
        d.write(InsertStatements)
generateInsertStatementsfromtext('user_id')
stop = time.time()
print("diff "+str(stop-start))
a = open("insertsfromtext.txt", 'r')# open file in read mode 
print("the file contains:") 
print(a.read())

## 4. 
### Export all three tables (Tweet, User and Geo tables) from the database into a |-separated text file (each value in a row should be separated by |). You do not generate SQL INSERT statements, just raw |-separated text data.

### a.	For the Geo table, add a new column with relative distance from a fixed point which is the location of CDM (41.878668, -87.625555). 
    You can simply treat it as a point-to-point Euclidean distance (although bonus points for finding a real distance in miles) and round the longitude and latitude columns to a maximum of 4 digits after the decimal.


In [ ]:
start= time.time()
f = open('geo.txt', 'w')
s = '|'
for row in cfile.execute('select * from geo').fetchall():
    a = row[0]
    b = row[1]
    c = float(row[2])
    d= float(row[3])
    diff = round(((41.878668-c)**2+(-87.625555-d)**2)**(1/2),4)
    f.write(str(a)+s+b+s+str(c)+s+str(d)+s+str(diff)+ '\n')
f.close()
stop = time.time()
print("diff "+str(stop-start))
a = open('geo.txt', 'r')
print("the file contains:") 
print(a.read())

### b. For the Tweet table, add two new columns from the User table (“name” and “screen_name”) in addition to existing columns.

In [ ]:
start= time.time()
f = open('tweet.txt', 'w')
for row in cfile.execute('select tweet.*, name, screen_name from tweet, user_id where tweet.user_id = user_id.id').fetchall():
    f.write(   str(row)[1:-1].replace(', ', '|')  + '\n' )
f.close()
stop = time.time()
print("diff "+str(stop-start))
a = open('tweet.txt', 'r')
print("the file contains:") 
print(a.read())

### c. For the User table file add a column that specifies how many tweets by that user are currently in the database. That is, your output file should contain all of the columns from the User table, plus the new column with tweet count.
    You do not need to modify the User table, just create the output text file. What is the name of the user with most tweets?

In [ ]:
start= time.time()
f=open('user.txt', 'w')
for row in cfile.execute('''select user_id.*, count(tweet.user_id) 
                            from user_id,tweet 
                            where tweet.user_id =user_id.id 
                            Group by tweet.user_id
                            having count(tweet.user_id) ''').fetchall():
    
    f.write(str(row)[1:-1].replace(', ', '|')  + '\n')
f.close()
stop = time.time()
print("diff "+str(stop-start))
a = open('user.txt', 'r')
print("the file contains:") 
print(a.read())